In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import calendar

In [4]:
#paths to kiki and will data 
k_disc_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\kiki_disc"
k_chase_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\kiki_chase"
k_ally_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\kiki_ally"
k_end_path = r"C:\Users\willd\OneDrive\Desktop\Budget\clean_data\k_data\k_budget_data.xlsx"

In [5]:
# combine all csvs in a path return df  // with no duplicates 
def combine_csvs(path):
    df_list = []
    for item in os.listdir(path):
        full_path = os.path.join(path, item)
        df = pd.read_csv(full_path)
        df_list.append(df)
    combined = pd.concat(df_list)
    combined.columns.values[0] = "Date"
    no_dup = combined.drop_duplicates()
    return no_dup


# remove specific payments from a df
def remove_item(df, descript):
    remove_df = df[~df['Description'].str.contains(descript)]
    return remove_df
    
    
# reformat to only date amount desc, category columns
def column_remove(df):
    df.columns = df.columns.str.replace(' ', '') 
    if 'Category' not in df.columns:
        df["Category"] = 'misc'
    new = df[['Date', 'Amount', 'Description', 'Category']]
    return new

# assign transaction catagory
def category_assign(df):
    # make seriess 
    s1 = df['Description']
    s2 = df['Amount']
    #ks category conditions and values
    k_cat_condition = [(s1.str.contains('VENMO'))&(s2 < (-600)),(s1.str.contains('VENMO')),
                       (s1.str.contains('COMCAST')),
                       (s1.str.contains('Primco')), (s1.str.contains('SCHWAB')), (s1.str.contains('Interest Paid'))]
    k_cat_value = ['RENT','VENMO','Bills & Utilities', 'Prim and Co', 'Investment', 'Intrest']
    df['Category'] = np.select(k_cat_condition, k_cat_value)
    df.loc[df['Category'] == '0', 'Category'] = 'Misc'
    return df 

In [17]:
# # ws categories conditions and values
# w_cat_condition = [(df.str.contains('VENMO')) & (df2 > 0) & (df2 < 600),(df.str.contains('VENMO'))& (df2 > 0) & (df2 > 600), (df.str.contains('VENMO')) & (df2 < 0), (df.str.contains('SPOTIFY') | df.str.contains('APPLE')),
#                  (df.str.contains('ACTALENT')), (df.str.contains('CHECK'))
#                 , (df.str.contains('SCHWAB BROK'))]
# w_cat_value = ['VENMO IN','KIKI RENT','VENMO OUT', 'SUBSCRIP', 'Belair', 'RENT', 'SCHWAB']
# s1 = df['Description']
# s2 = df['Amount']
# k_cat_condition = [(s1.str.contains('VENMO'))& (s2 < 600),(s1.str.contains('VENMO'))&(s2 < -600),
#                    (s1.str.contains('COMCAST')),
#                    (s1.str.contains('Primco')), (s1.str.contains('SCHWAB')), (s1.str.contains('Interest Paid'))]
# k_cat_value = ['VENMO','RENT', 'Bills & Utilities', 'Prim and Co', 'Investment', 'Intrest']

In [6]:
k_disc = column_remove(combine_csvs(k_disc_path))
k_chase = column_remove(combine_csvs(k_chase_path))
k_ally = column_remove(combine_csvs(k_ally_path))
# categorize ally transactions
k_ally = category_assign(k_ally)
#remove cc payments from ally
ally_no_cc = remove_item(k_ally, 'CHASE CREDIT CRD')
chase_no_cc = remove_item(k_chase, 'AUTOMATIC PAYMENT')
disc_no_cc = remove_item(k_disc, 'DIRECTPAY FULL BALANCESEE DETAILS')
# make remoining discover 
disc_no_cc['Amount'] = 0 - disc_no_cc['Amount']

C:\Users\willd\AppData\Local\Temp\ipykernel_22980\2493645993.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disc_no_cc['Amount'] = 0 - disc_no_cc['Amount']


In [8]:
full_df =  pd.concat([disc_no_cc, chase_no_cc, ally_no_cc])

In [9]:
# change date dtype
full_df['Date'] = pd.to_datetime(full_df['Date'])
full_df['Month'] = pd.DatetimeIndex(full_df['Date']).month
# change month colum to date
full_df['Month'] = full_df['Month'].apply(lambda x: calendar.month_abbr[x])
full_df = full_df.sort_values(by='Date')

In [10]:
full_df = full_df[full_df['Date'] >= '2023-09-01']

In [11]:
money_in = full_df[full_df['Amount'] > 0]
money_out = full_df[full_df['Amount'] < 0]
money_out.loc[money_out['Amount'] < 0, 'Amount'] = 0 - money_out['Amount']


# df of expense to income per month 
month_list = full_df['Month'].unique()
dict_list = []
for mon in month_list:
    df_in = money_in[money_in['Month'] == mon]
    df_out = money_out[money_out['Month'] == mon]
    month_in = df_in['Amount'].sum()
    month_out = df_out['Amount'].sum()
    dict1 = {'Month': mon, 'Money In': month_in, 'Money Out': month_out}
    dict_list.append(dict1)
monthly_ratio = pd.DataFrame(dict_list)

C:\Users\willd\AppData\Local\Temp\ipykernel_22980\3061870372.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  money_out.loc[money_out['Amount'] < 0, 'Amount'] = 0 - money_out['Amount']


In [12]:
writer = pd.ExcelWriter(k_end_path, engine="xlsxwriter")
full_df.to_excel(writer, sheet_name="full_df")
monthly_ratio.to_excel(writer, sheet_name="all_mon")
money_in.to_excel(writer, sheet_name="income")
money_out.to_excel(writer, sheet_name="expense")
writer.close()